# The Max-cut problem
 
This notebook demonstrates a concrete example of Quadratic Unconstrained Binary Optimization (QUBO) in the case of the Max-Cut problem. Given a graph, the objective of the Max-Cut problem is to separate all nodes into two sets such that the number of edges between these sets is maximized. Max-Cut can be formulated as a QUBO problem.


In [ ]:
# First, perform the relevant imports and navigate to the root folder
import os
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

if os.getcwd()[-7:]=="max_cut":
    os.chdir("../..")

from quantumqubo.qubo import QUBO
from applications_notebooks.max_cut.utils_max_cut import separate_in_two_subsets


### Toy instance of Max-Cut

We select a simple graph with 5 nodes on which to demonstrate this application of ORCA's QUBO solver.

In [ ]:
G = nx.Graph()
edge_list = [(0,1), (1,3), (2,4), (3,2), (0,2), (3,4)]

G.add_edges_from(edge_list)
pos = nx.spring_layout(G, seed=7)  # positions for all nodes - seed for reproducibility
nx.draw_networkx_nodes(G, pos, node_size=700, node_color='blue')
nx.draw_networkx_edges(G, pos, edgelist=edge_list, width=5)
nx.draw_networkx_labels(G, pos, font_size=20, font_family="sans-serif")

ax = plt.gca()
ax.margins(0.08)
plt.axis("off")
plt.show()

### Draw a solution for this instance

In [ ]:
separate_in_two_subsets(G, [1,2], [0,3,4])

# Solving this instance with ORCA's QUBO solver

### Max-Cut formulation as a QUBO problem

We introduce the variable $x_i = $

$$
x_i = \left\{
    \begin{array}{ll}
        0 \quad \text{if is in subset A} \\
        1 \quad \text{if is in subset B}
    \end{array}
\right.
$$

Then,

$$
x_i + x_j -2x_i x_j = \left\{
    \begin{array}{ll}
        0 \quad \text{if} \quad(x_i, x_j) \quad \text{are in the same subset} \\
        1 \quad \text{if} \quad(x_i, x_j) \quad \text{are in different subset}
    \end{array}
\right.
$$

Solving Max-Cut is thus equivalent to solving the following QUBo problem:
$$
C = \sum_{(x_i,x_j) \in E} x_i + x_j -2x_i x_j
$$

In [ ]:
# Define the matrix Q used for QUBO

Q = np.zeros((5,5))
for (i,j) in G.edges:
    Q[i,i] += -1
    Q[j,j] += -1
    Q[i,j] += 1
    Q[j,i] += 1
                
print(Q)

### Run the training algorithm

In [ ]:
def qubo_function(vect):
    return np.dot(vect, np.dot(Q, vect))

qubo = QUBO(
    5,
    qubo_function
)

qubo.train(
    learning_rate=1e-1,
    updates=40,
    samples_per_point=20,
    print_frequency=20
    )

### Draw the solution

In [ ]:
solution = list(qubo.res.keys())[0]
nodes_in_set1 = [idx for idx in range(len(solution)) if solution[idx]==0]
nodes_in_set2 = [idx for idx in range(len(solution)) if solution[idx]==1]

separate_in_two_subsets(G, nodes_in_set1, nodes_in_set2)